In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/data (1).csv")
from google.colab import files
import pandas as pd

# Assuming your DataFrame is named 'data'

# Define a mapping dictionary
sentiment_mapping = {"positive": 0, "Positive": 0, "negative": 1, "Negative": 1, "neutral": 2, "Neutral": 2}

# Create a new column named 'sentiment_numeric' using the mapping dictionary
data['sentiment_numeric'] = data['sentiment'].map(sentiment_mapping)





In [ ]:
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
model = TFAutoModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
text_list = data['text'].tolist()
inputs = tokenizer(text_list,padding=True, truncation =True, return_tensors='tf')

In [ ]:
output = model(inputs)
output

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset, Features, Value, ClassLabel,DatasetDict

# Assuming your DataFrame is named 'data'from datasets import Features, Value, ClassLabel

# Define the schema for the dataset
features = Features({
    'text': Value('string'),
    'sentiment': ClassLabel(names=['negative', 'positive','neutral','Positive','Negative','Neutral']),
    'sentiment_numeric': Value('int64')
})

# Create the dataset from the DataFrame
dataset = Dataset.from_pandas(data, features=features)


In [ ]:
data.columns

In [ ]:
data = data.reset_index(drop=False)  # Add existing index as a new column 'index'
data.head()


,index,sentiment,text,sentiment_numeric
0,0,positive,The cybersecurity team successfully thwarted a...,0
1,1,negative,Users are distressed as a malevolent data brea...,1
2,2,neutral,The system underwent intense scrutiny during s...,2
3,3,positive,Joyful employees celebrate the implementation ...,0
4,4,negative,Customers express deep dissatisfaction as a de...,1


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets with 20% for testing
train_test, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=42)

# Further split the training set into training and validation sets with 10% for validation
train, val = train_test_split(train_test, test_size=0.1, shuffle=True, random_state=42)

# Extract training, validation, and testing sets
train_data, val_data, test_data = train, val, test


In [ ]:
# Convert dataframes to datasets
import datasets
train_dataset = datasets.Dataset.from_pandas(train_data)
test_dataset = datasets.Dataset.from_pandas(test_data)
validation_dataset = datasets.Dataset.from_pandas(val_data)

# Create a DatasetDict
data_dict = datasets.DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": validation_dataset
})

In [ ]:
text_encoded = data_dict.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
text_encoded

In [ ]:
import tensorflow as tf

In [ ]:
text_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'sentiment_numeric'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]
# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(text_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(text_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 38), dtype=int64, numpy=
array([[ 101, 1000, 1996, ...,    0,    0,    0],
       [ 101, 1000, 1996, ...,    0,    0,    0],
       [ 101, 1000, 1996, ...,    0,    0,    0],
       ...,
       [ 101, 1000, 1996, ..., 1012, 1000,  102],
       [ 101, 1000, 1996, ...,  102,    0,    0],
       [ 101, 5936, 4125, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(64, 38), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 38), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[0 1 0 2 2 2 2 1 0 0 2 2 2 1 0 0 2 2 2 1 0 1 2 2 1 2 0 1 0 1 2 

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
classifier = BERTForClassification(model, num_classes=6)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = classifier.fit(
    train_dataset,
    epochs=10
)

Epoch 1/10
2/2 [==============================] - 27s 4s/step - loss: 1.0722 - accuracy: 0.7917
Epoch 2/10
2/2 [==============================] - 27s 23s/step - loss: 0.9497 - accuracy: 0.8056
Epoch 3/10
2/2 [==============================] - 27s 4s/step - loss: 0.9005 - accuracy: 0.7917
Epoch 4/10
2/2 [==============================] - 27s 23s/step - loss: 0.8054 - accuracy: 0.8056
Epoch 5/10
2/2 [==============================] - 27s 23s/step - loss: 0.7481 - accuracy: 0.8056
Epoch 6/10
2/2 [==============================] - 27s 4s/step - loss: 0.6890 - accuracy: 0.8194
Epoch 7/10
2/2 [==============================] - 27s 23s/step - loss: 0.5912 - accuracy: 0.8194
Epoch 8/10
2/2 [==============================] - 27s 23s/step - loss: 0.5246 - accuracy: 0.8889
Epoch 9/10
2/2 [==============================] - 27s 4s/step - loss: 0.4859 - accuracy: 0.9306
Epoch 10/10
2/2 [==============================] - 27s 4s/step - loss: 0.4461 - accuracy: 0.9722


In [ ]:
classifier.evaluate(test_dataset)

1/1 [==============================] - 12s 12s/step - loss: 0.9314 - accuracy: 0.8000


[0.9313982129096985, 0.800000011920929]